## Imports

In [43]:
import pandas as pd
import numpy as np
import pyswarms as ps
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import os
import pyswarms as ps
from warnings import simplefilter, filterwarnings

filterwarnings("ignore", category=FutureWarning)
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

## Parameters

In [66]:
features = ['simple_sugars', 'complex_sugars', 'fats', 'dietary_fibers', 'proteins', 'fast_insulin', 'slow_insulin']
meal_features = ['simple_sugars', 'complex_sugars', 'fats', 'dietary_fibers', 'proteins']
insulin_features = ['fast_insulin', 'slow_insulin']
features_to_remove = ['glucose_next', 'datetime', 'patient']

patients = ['001', '002', '004', '006', '007', '008']
approaches = ['pixtral-large-latest', 'nollm']
prediction_horizons = [6, 12]

train_size = 0.95
n_particles = 10
iters = 10

lgb_params = {
    'max_depth': 3,
    'n_estimators': 1000, 
    'learning_rate': 0.1,
    'objective': 'regression',
    'data_sample_strategy': 'goss',
    'random_state': 1,
    'deterministic': True,
    'use_quantized_grad': True,
    'force_row_wise': True,
    'num_threads': 5,
    'verbosity': -1,
}

model = lgb.LGBMRegressor(**lgb_params)
callbacks = [lgb.early_stopping(stopping_rounds=5, verbose=False)]  

patient_weight = 5
prior_hour_weight = 10

## Functions

In [67]:
def get_projected_value(window, prediction_horizon):
    x = np.arange(len(window))
    coeffs = np.polyfit(x, window, deg=3)
    poly = np.poly1d(coeffs)
    projected_value = poly(len(window) + prediction_horizon)
    return projected_value

def get_data(patient, prediction_horizon):
    glucose_data = pd.read_csv(f"diabetes_subset_pictures-glucose-food-insulin/{patient}/glucose.csv")
    insulin_data = pd.read_csv(f"diabetes_subset_pictures-glucose-food-insulin/{patient}/insulin.csv")
    food_data = pd.read_csv(f"food_data/pixtral-large-latest/{patient}.csv")

    glucose_data["datetime"] = pd.to_datetime(glucose_data["date"] + ' ' + glucose_data["time"])
    glucose_data.drop(['type', 'comments', 'date', 'time'], axis=1, inplace=True)
    glucose_data['glucose'] *= 18.0182  

    insulin_data["datetime"] = pd.to_datetime(insulin_data["date"] + ' ' + insulin_data["time"])
    insulin_data.drop(['comment', 'date', 'time'], axis=1, inplace=True)

    food_data['datetime'] = pd.to_datetime(food_data['datetime'], format='%Y:%m:%d %H:%M:%S')
    food_data = food_data[['datetime', 'simple_sugars', 'complex_sugars', 'proteins', 'fats', 'dietary_fibers']]

    combined_data = pd.concat([food_data, insulin_data]).sort_values('datetime').reset_index(drop=True)
    combined_data.fillna(0, inplace=True)
    glucose_data['hour'] = glucose_data['datetime'].dt.hour

    glucose_data['glucose_next'] = glucose_data['glucose'] - glucose_data['glucose'].shift(-prediction_horizon)

    glucose_data['glucose_change'] = glucose_data['glucose'] - glucose_data['glucose'].shift(1)

    glucose_data['glucose_change_projected'] = glucose_data['glucose_change'].rolling(
        window=6, min_periods=6
    ).apply(lambda window: get_projected_value(window, prediction_horizon))
    glucose_data['glucose_projected'] = glucose_data['glucose'].rolling(
        window=6, min_periods=6
    ).apply(lambda window: get_projected_value(window, prediction_horizon))
    glucose_data.dropna(subset=['glucose_next'], inplace=True)
    return glucose_data, combined_data

def add_features(params, features, preprocessed_data, prediction_horizon):
    patients_glucose_data = []
    
    for patient in preprocessed_data.keys():
        glucose_data, combined_data = preprocessed_data[patient]
        
        glucose_times = glucose_data['datetime'].values.astype('datetime64[s]').astype(np.int64)
        combined_times = combined_data['datetime'].values.astype('datetime64[s]').astype(np.int64)
        
        for feature in features:
                
            metabolism_rate, peak_time = params[feature]
            
            feature_values = combined_data.loc[:, feature].values
            
            time_diff_hours = ((glucose_times[:, None] - combined_times[None, :]) / 3600)
            weights = np.zeros_like(time_diff_hours)
            
            # Increasing phase
            increase_mask = (time_diff_hours >= 0) & (time_diff_hours < peak_time)
            weights[increase_mask] = time_diff_hours[increase_mask] / peak_time
            
            # Decreasing phase
            decrease_mask = time_diff_hours >= peak_time
            weights[decrease_mask] = 1 - ((time_diff_hours[decrease_mask] - peak_time) * metabolism_rate)
            
            weights = np.clip(weights, 0, None)
            glucose_data[feature] = np.dot(weights, feature_values)
            glucose_data[feature] = glucose_data[feature].shift(-prediction_horizon)

        glucose_data['patient'] = patient
        patients_glucose_data.append(glucose_data)
        
    patients_glucose_data = pd.concat(patients_glucose_data)
    patients_glucose_data.dropna(inplace=True)
    return patients_glucose_data

def train_and_evaluate_by_hour(patient, hour, model, train, test, features_to_remove, callbacks, train_size, patient_weight, prior_hour_weight):
    """Train model for specific hour and evaluate on test data"""
    weights = np.ones(len(train))
    patient_mask = train['patient'] == patient
    prior_hour_mask = train['hour'] < hour
    weights[patient_mask] = patient_weight
    weights[patient_mask & prior_hour_mask] = prior_hour_weight
    X_test = test.drop(features_to_remove, axis=1)
    X_train, X_val, y_train, y_val, weights_train, weights_val = train_test_split(
        train.drop(features_to_remove, axis=1), train['glucose_next'], weights, train_size=train_size, random_state=42
    )
    model.fit(X_train, y_train,
              sample_weight=weights_train,
              eval_set=[(X_val, y_val)], 
              eval_sample_weight=[weights_val],
              callbacks=callbacks)
    
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(test['glucose_next'], preds))
    return rmse, preds

In [68]:
train_size = 0.95

lgb_params = {
    'max_depth': 3,
    'n_estimators': 1000, 
    'learning_rate': 0.1,
    'reg_lambda': 10,
    'objective': 'regression',
    'random_state': 1,
    'deterministic': True,
    'data_sample_strategy': 'goss',
    'num_threads': 5,
    'use_quantized_grad': True,
    'verbosity': -1
}

model = lgb.LGBMRegressor(**lgb_params)
callbacks = [lgb.early_stopping(stopping_rounds=10, verbose=False)]  

feature_params = {
    'simple_sugars': [0.5, 0.5], 
    'complex_sugars': [0.3, 0.5], 
    'proteins': [0.2, 4], 
    'fats': [0.1, 3], 
    'dietary_fibers': [0.05, 3.5], 
    'fast_insulin': [0.5, 0.5], 
    'slow_insulin': [0.5, 1]
}

In [69]:
def optimize_patient_parameters(patients, prediction_horizon, base_feature_params):
    """Optimize feature parameters for each patient using first 3 days of data"""
    patient_feature_params = {}
    
    for patient in patients:
        # Get data for all patients
        all_data = {p: get_data(p, prediction_horizon) for p in patients}
        
        # Extract first 3 days for training/optimization
        training_data = {}
        for p in patients:
            glucose_data, combined_data = all_data[p]
            if p == patient:
                # Get first 3 days of data for target patient
                first_days = glucose_data['datetime'].dt.day.unique()[:3]
                mask = glucose_data['datetime'].dt.day.isin(first_days)
                training_data[p] = (glucose_data[mask], combined_data)
            else:
                training_data[p] = (glucose_data, combined_data)
        
        # Define bounds for optimization
        lb = []  # Lower bounds
        ub = []  # Upper bounds
        
        for feature in features:
            base_values = base_feature_params[feature]
            # Add metabolism rate bounds
            lb.append(max(0.2, 0.2 * base_values[0]))
            ub.append(min(5.0, 5.0 * base_values[0]))
            
            # Add peak time bounds
            lb.append(max(0.2, 0.2 * base_values[1]))
            ub.append(min(5.0, 5.0 * base_values[1]))
            
            # Add sensitivity bounds
            lb.append(0.2)
            ub.append(5.0)
        
        bounds = (np.array(lb), np.array(ub))
        
        # Define objective function for PSO
        def objective_function(params):
            rmse_scores = []
            for pos in range(params.shape[0]):
                particle = params[pos]
                # Convert parameters to dictionary with sensitivity applied
                feature_params_dict = {}
                sensitivity_dict = {}
                
                for i, feature in enumerate(features):
                    idx = i * 3
                    metabolism_rate = particle[idx]
                    peak_time = particle[idx + 1]
                    sensitivity = particle[idx + 2]
                    
                    feature_params_dict[feature] = [metabolism_rate, peak_time]
                    sensitivity_dict[feature] = sensitivity
                
                # Process data with these parameters
                processed_data = add_features_with_sensitivity(feature_params_dict, sensitivity_dict, features, training_data, prediction_horizon)
                processed_data = processed_data[processed_data['patient'] == patient]
                
                # Split features and target
                X = processed_data.drop(features_to_remove, axis=1)
                y = processed_data['glucose_next']
                X_train, X_val, y_train, y_val = train_test_split(
                    X, y, train_size=train_size, random_state=42
                )
                
                # Simple model prediction (avoiding full training for speed)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
                preds = model.predict(X_val)
                rmse = np.sqrt(mean_squared_error(y_val, preds))
                rmse_scores.append(rmse)
                
            return np.array(rmse_scores)
        
        # Initialize swarm
        options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
        optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, 
                                          dimensions=len(features)*3,
                                          options=options, 
                                          bounds=bounds)
        
        # Optimize
        cost, pos = optimizer.optimize(objective_function, iters=iters, verbose=False)
        
        # Extract best parameters
        best_params = {}
        for i, feature in enumerate(features):
            idx = i * 3
            metabolism_rate = pos[idx]
            peak_time = pos[idx + 1]
            sensitivity = pos[idx + 2]
            best_params[feature] = [metabolism_rate, peak_time, sensitivity]
        
        patient_feature_params[patient] = best_params
        print(f"Completed optimization for patient {patient}, best RMSE: {cost:.4f}")
    
    return patient_feature_params

def add_features_with_sensitivity(params, sensitivity_dict, features, preprocessed_data, prediction_horizon):
    """Enhanced version of add_features that considers sensitivity"""
    patients_glucose_data = []
    
    for patient in preprocessed_data.keys():
        glucose_data, combined_data = preprocessed_data[patient]
        
        glucose_times = glucose_data['datetime'].values.astype('datetime64[s]').astype(np.int64)
        combined_times = combined_data['datetime'].values.astype('datetime64[s]').astype(np.int64)
        
        for feature in features:
                
            metabolism_rate, peak_time = params[feature]
            sensitivity = sensitivity_dict[feature]
            
            # Apply sensitivity to affect the impact of this feature
            feature_values = combined_data.loc[:, feature].values * sensitivity
            
            time_diff_hours = ((glucose_times[:, None] - combined_times[None, :]) / 3600)
            weights = np.zeros_like(time_diff_hours)
            
            # Increasing phase
            increase_mask = (time_diff_hours >= 0) & (time_diff_hours < peak_time)
            weights[increase_mask] = time_diff_hours[increase_mask] / peak_time
            
            # Decreasing phase
            decrease_mask = time_diff_hours >= peak_time
            weights[decrease_mask] = 1 - ((time_diff_hours[decrease_mask] - peak_time) * metabolism_rate)
            
            weights = np.clip(weights, 0, None)
            glucose_data[feature] = np.dot(weights, feature_values)

            glucose_data[feature] = glucose_data[feature].shift(-prediction_horizon)

        glucose_data['patient'] = patient
        patients_glucose_data.append(glucose_data)
        
    patients_glucose_data = pd.concat(patients_glucose_data)
    patients_glucose_data.dropna(inplace=True)
    return patients_glucose_data

## Loop

In [70]:
# Get patient-specific parameters first
prediction_horizons_to_optimize = [6, 12]  # Optimize for both prediction horizons
patient_params = {}

for prediction_horizon in prediction_horizons_to_optimize:
    print(f"Optimizing parameters for prediction horizon {prediction_horizon}")
    patient_params[prediction_horizon] = optimize_patient_parameters(patients, prediction_horizon, feature_params)

Optimizing parameters for prediction horizon 6
Completed optimization for patient 001, best RMSE: 9.8218
Completed optimization for patient 002, best RMSE: 14.0012
Completed optimization for patient 004, best RMSE: 16.0421
Completed optimization for patient 006, best RMSE: 13.4540
Completed optimization for patient 007, best RMSE: 7.6162
Completed optimization for patient 008, best RMSE: 10.3974
Optimizing parameters for prediction horizon 12
Completed optimization for patient 001, best RMSE: 15.4936
Completed optimization for patient 002, best RMSE: 23.7349
Completed optimization for patient 004, best RMSE: 22.7516
Completed optimization for patient 006, best RMSE: 21.4533
Completed optimization for patient 007, best RMSE: 13.9024
Completed optimization for patient 008, best RMSE: 19.4252


Optimizing parameters for prediction horizon 6
Completed optimization for patient 001, best RMSE: 15.2413
Completed optimization for patient 002, best RMSE: 15.4925
Completed optimization for patient 004, best RMSE: 17.9196
Completed optimization for patient 006, best RMSE: 16.0420
Completed optimization for patient 007, best RMSE: 20.8546
Completed optimization for patient 008, best RMSE: 18.0485
Optimizing parameters for prediction horizon 12
Completed optimization for patient 001, best RMSE: 27.8845
Completed optimization for patient 002, best RMSE: 37.8872
Completed optimization for patient 004, best RMSE: 33.4583
Completed optimization for patient 006, best RMSE: 34.7633
Completed optimization for patient 007, best RMSE: 31.2486
Completed optimization for patient 008, best RMSE: 30.7365

In [73]:
patient_params

{6: {'001': {'simple_sugars': [1.1618128212337295,
    1.7198326301028943,
    3.4062057491841693],
   'complex_sugars': [1.0397125140597963,
    1.4648114608393286,
    4.174684245937805],
   'fats': [0.3233505140941043, 3.0777345501567535, 2.6253803508482276],
   'dietary_fibers': [0.2079360538964481,
    3.1810789932000163,
    3.3670427519118196],
   'proteins': [0.6690410982598018, 3.5848529020576607, 4.59760155237653],
   'fast_insulin': [2.2241802927295744,
    0.46534604719066924,
    2.9906658003057616],
   'slow_insulin': [2.125303345177023, 4.091231494551917, 4.858616498291669]},
  '002': {'simple_sugars': [1.7606887350435938,
    1.3430269816348066,
    3.9585114804657664],
   'complex_sugars': [1.2118028304484492,
    1.2321217986249797,
    3.2513096919909867],
   'fats': [0.37009757932026793, 3.26455501229759, 2.593822674415963],
   'dietary_fibers': [0.23179752749479055,
    3.1432889692556705,
    1.6850994394873569],
   'proteins': [0.7309588954292894, 2.3998045174048

In [75]:
patient_optimized_params[feature]

[1.1618128212337295, 1.7198326301028943, 3.4062057491841693]

In [77]:
# Main evaluation loop
df = pd.DataFrame(columns=['Approach', 'Prediction Horizon', 'Patient', 'Day', 'Hour', 'RMSE']) 

for approach in approaches:
    for prediction_horizon in prediction_horizons:
        # Get raw data for all patients
        data = {patient: get_data(patient, prediction_horizon) for patient in patients}
        all_processed_data = []
        
        for patient in patients:
            # Get optimized parameters for this patient/horizon
            patient_optimized_params = patient_params[prediction_horizon][patient]
            
            # Prepare feature parameters and sensitivity dictionaries
            feature_params_dict = {}
            sensitivity_dict = {}
            
            # Select which features based on approach
            features_to_process = features.copy() if approach == 'pixtral-large-latest' else insulin_features.copy()
            
            # Extract parameter values for selected features
            for feature in features_to_process:
                metabolism_rate, peak_time, sensitivity = patient_optimized_params[feature]
                feature_params_dict[feature] = [metabolism_rate, peak_time]
                sensitivity_dict[feature] = sensitivity
            
            # Process just this patient's data
            processed_data = add_features_with_sensitivity(
                feature_params_dict, 
                sensitivity_dict,
                features_to_process, 
                {patient: data[patient]}, 
                prediction_horizon
            )
            all_processed_data.append(processed_data)
        
        # Combine all processed data
        processed_data = pd.concat(all_processed_data)
        
        # Process per patient for evaluation
        for patient in patients:
            patient_data = processed_data[processed_data['patient'] == patient]
            all_preds = []
            all_test_data = []
            
            days = patient_data['datetime'].dt.day.unique()
            for day in days[3:]:  # Skip first 3 days used for optimization
                day_mask = patient_data['datetime'].dt.day == day
                test_day = patient_data[day_mask]
                other_days = patient_data[~day_mask]
                
                for hour in test_day['hour'].unique():
                    hour_model = lgb.LGBMRegressor(**lgb_params)
                    hour_mask = test_day['hour'] == hour
                    test = test_day[hour_mask]
                    prior_hours_mask = (test_day['hour'] < hour) & (test_day['datetime'].dt.day == day)
                    train = pd.concat([test_day[prior_hours_mask], other_days])
                    
                    hour_rmse, hour_preds = train_and_evaluate_by_hour(
                        patient, hour, hour_model, train, test, 
                        features_to_remove, callbacks, train_size=0.95, 
                        patient_weight=patient_weight, prior_hour_weight=prior_hour_weight
                    )
                    
                    all_preds.append(hour_preds)
                    all_test_data.append(test)
                    
                    df = pd.concat([df, pd.DataFrame({
                        'Approach': [approach],
                        'Prediction Horizon': [prediction_horizon],
                        'Patient': [patient],
                        'Day': [day],
                        'Hour': [hour],
                        'RMSE': [hour_rmse]
                    })], ignore_index=True)


            combined_preds = np.concatenate(all_preds)
            combined_test = pd.concat(all_test_data)
            
            predictions = pd.DataFrame({
                'Predictions': combined_test['glucose'] - combined_preds, 
                'Ground_truth': combined_test['glucose'] - combined_test['glucose_next'], 
                'Datetime': combined_test['datetime']
            })
            
            os.makedirs(f'predictions/{approach}/{prediction_horizon}', exist_ok=True)
            predictions.to_csv(f'predictions/{approach}/{prediction_horizon}/{patient}_predictions.csv', index=False)
        
        # Save processed data
        os.makedirs('datasets', exist_ok=True)
        processed_data.to_csv(f'datasets/{approach}_{prediction_horizon}.csv', index=False)
            
        # Calculate and report average RMSE
        current_rmse = df[(df['Approach'] == approach) & (df['Prediction Horizon'] == prediction_horizon)]['RMSE'].mean()
        print(f"Average RMSE for {approach}, prediction horizon {prediction_horizon}: {current_rmse:.4f}")

Average RMSE for pixtral-large-latest, prediction horizon 6: 15.9558
Average RMSE for pixtral-large-latest, prediction horizon 12: 30.9849
Average RMSE for nollm, prediction horizon 6: 15.9432
Average RMSE for nollm, prediction horizon 12: 31.4015
